# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
weather_df = pd.read_csv("../weather_data_df.csv", encoding="utf-8")
del weather_df["Unnamed: 0"]
weather_df.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
#Configure gmaps
gmaps.configure(api_key=g_key)

locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)
max_humidity = humidity.max()

In [ ]:
fig = gmaps.figure(center=(30.0, -10.0), zoom_level=2)
heat_map = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max_humidity, point_radius=2)
fig.add_layer(heat_map)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
ideal_df = weather_df.loc[(weather_df["Temperature"] > 70) & (weather_df["Temperature"] <80) & (weather_df["Wind Speed"] < 3), :]
ideal_df = ideal_df.dropna(how="any")

ideal_df.reset_index(inplace=True)
del ideal_df["index"]

ideal_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
#Create hotel list
hotel_list = []

#Create loop for hotel list
for city in range(len(ideal_df)):
    lat = ideal_df.loc[city]["Lat"]
    lng = ideal_df.loc[city]["Lng"]
    
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types": "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    jsn = response.json()
    try:
        hotel_list.append(jsn["results"][0]["name"])
    except:
        hotel_list.append("")

ideal_df["Hotel Name"] = hotel_list

ideal_df = ideal_df.dropna(how="any")
ideal_df.head()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = ideal_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig
